# Encontrando a palavra certa

Aqui vamos utilizar o PLN para tentar prever qual será a próxima palavra de um soneto do Shakespeare. 

In [1]:
# Baixando a base de dados

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
  
data = open('/tmp/sonnets.txt').read()
corpus = data.lower().split("\n")

--2020-08-20 12:43:35--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.69.128, 173.194.79.128, 108.177.126.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2020-08-20 12:43:35 (135 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.utils as ku 
import tensorflow as tf
import numpy as np 

In [3]:
# Limpando sessão e setando semente

tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index)+1
dict(list( tokenizer.word_index.items())[0:10])

{'and': 1,
 'i': 6,
 'in': 7,
 'my': 5,
 'of': 4,
 'that': 8,
 'the': 2,
 'thou': 10,
 'thy': 9,
 'to': 3}

In [5]:
# Criando sequences a partir de dos tokens
input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

max_sequence_len = max(len(x) for x in input_sequences)
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

In [6]:
model = Sequential()

In [7]:
model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_len-1))

In [8]:
model.add(Bidirectional(LSTM(150, return_sequences = True)))

In [9]:
model.add(Dropout(0.2))

In [10]:
model.add(LSTM(100))

In [11]:
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

In [12]:
model.add(Dense(total_words, activation='softmax'))

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 100)           321100    
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 300)           301200    
_________________________________________________________________
dropout (Dropout)            (None, 10, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 1605)              162105    
_________________________________________________________________
dense_1 (Dense)              (None, 3211)              5156866   
Total params: 6,101,671
Trainable params: 6,101,671
Non-trainable params: 0
______________________________________________

In [14]:
earlystop = EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=0, mode='auto')
history = model.fit(predictors, label, epochs=500, verbose=1, callbacks=[earlystop])

Epoch 1/500
484/484 [==============================] - 6s 12ms/step - loss: 6.9060 - accuracy: 0.0213
Epoch 2/500
484/484 [==============================] - 6s 11ms/step - loss: 6.4992 - accuracy: 0.0213
Epoch 3/500
484/484 [==============================] - 6s 12ms/step - loss: 6.3860 - accuracy: 0.0244
Epoch 4/500
484/484 [==============================] - 6s 11ms/step - loss: 6.2643 - accuracy: 0.0297
Epoch 5/500
484/484 [==============================] - 5s 11ms/step - loss: 6.1719 - accuracy: 0.0371
Epoch 6/500
484/484 [==============================] - 6s 11ms/step - loss: 6.0892 - accuracy: 0.0386
Epoch 7/500
484/484 [==============================] - 5s 11ms/step - loss: 6.0092 - accuracy: 0.0425
Epoch 8/500
484/484 [==============================] - 5s 11ms/step - loss: 5.9163 - accuracy: 0.0461
Epoch 9/500
484/484 [==============================] - 5s 11ms/step - loss: 5.8136 - accuracy: 0.0519
Epoch 10/500
484/484 [==============================] - 5s 11ms/step - loss: 5.703

In [19]:
print(model)

In [15]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [18]:
seed_text = "I've got a bad feeling about this"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list), axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

I've got a bad feeling about this were lose seeming masonry injury of state within thine days ears so shows me we bright now woe see amiss woe bright bright prove rolling days new rich do better skill add worth alone was me strong ' see ' untrue strange decays alone remember'd afloat oppress'd with melancholy oppress'd and ruinate indigest dote confounds me was woe respect nearly see eyes prove thee words and bad i come by kings scope wide night young age shall mother brought of sorrows growing age age re lips of days 'tis bring a confined doom and date fall of thee days confounds
